## Advanced Regression on Housing Data

#### Team:
1. Hoa Nguyen
2. Shailesh Krishna
3. Sheetal Kalburgi

#### Introduction
For this project, we will be applying the dimension reduction and clustering techniques on the dataset containing both numerical as well as categorical features. For this purpose we are using the well known [housing dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview) on Kaggle.

In [4]:
# Package imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
import prince
import gower
import itertools
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
import time
import pyclustering
from pyclustering.cluster.kmedoids import kmedoids
from sklearn.metrics.cluster import normalized_mutual_info_score

**Data Cleaning**  
* Removing the columns that contains too many missing values.
* Remove the missing values if any in the dataset.

In [5]:
# Load data set
data = pd.read_csv('train.csv')
data = data.drop('Id', axis = 1)

# Remove columns that have too many missing values
data = data.drop(data.columns[data.isnull().sum() > 30], axis = 1)

# Remove missing values
data.dropna(inplace = True)

In [6]:
print("The cleaned dataset contains {} rows and {} columns".format(data.shape[0],data.shape[1]))

The cleaned dataset contains 1451 rows and 64 columns


In [7]:
# Check if there are any missing values in the dataset
print("Are there null values in the cleaned dataset?: {}".format(data.isnull().values.any()))

Are there null values in the cleaned dataset?: False


### Part I: Dimension Reduction

**1. Define X and y**

In [8]:
X = data.drop(columns='SalePrice',axis=1)
y = data['SalePrice']

**2. Determine the unique data types**
* Determine the unique data types in the cleaned dataset to segregate the features.

In [9]:
# Data types
column_datatypes = set()
for column in data.columns:
    column_datatypes.add(str(data[column].dtype))
print("The cleaned dataset contains {} different data types and they are: {}".format(len(column_datatypes), ", ".join(column_datatypes)))

The cleaned dataset contains 3 different data types and they are: float64, int64, object


In [10]:
numerical_features = list()
categorical_features = list()
for column in X.columns:
    # In the dataset we only have float and int64.
    if (data[column].dtype == 'float64' or data[column].dtype == 'int64'):
        numerical_features.append(column)
    # Categorical
    elif (data[column].dtype == 'object'):
        categorical_features.append(column)

In [11]:
print('There are a total of {} numerical features in the dataset.'.format(len(numerical_features)))

There are a total of 34 numerical features in the dataset.


In [12]:
print('Numerical features in the dataset are:{}'.format(",  ".join(numerical_features)))

Numerical features in the dataset are:MSSubClass,  LotArea,  OverallQual,  OverallCond,  YearBuilt,  YearRemodAdd,  MasVnrArea,  BsmtFinSF1,  BsmtFinSF2,  BsmtUnfSF,  TotalBsmtSF,  1stFlrSF,  2ndFlrSF,  LowQualFinSF,  GrLivArea,  BsmtFullBath,  BsmtHalfBath,  FullBath,  HalfBath,  BedroomAbvGr,  KitchenAbvGr,  TotRmsAbvGrd,  Fireplaces,  GarageCars,  GarageArea,  WoodDeckSF,  OpenPorchSF,  EnclosedPorch,  3SsnPorch,  ScreenPorch,  PoolArea,  MiscVal,  MoSold,  YrSold


In [13]:
print('There are a total of {} categorical features in the dataset.'.format(len(categorical_features)))

There are a total of 29 categorical features in the dataset.


In [14]:
print('Categorical features in the dataset are: {}'.format(",  ".join(categorical_features)))

Categorical features in the dataset are: MSZoning,  Street,  LotShape,  LandContour,  Utilities,  LotConfig,  LandSlope,  Neighborhood,  Condition1,  Condition2,  BldgType,  HouseStyle,  RoofStyle,  RoofMatl,  Exterior1st,  Exterior2nd,  MasVnrType,  ExterQual,  ExterCond,  Foundation,  Heating,  HeatingQC,  CentralAir,  Electrical,  KitchenQual,  Functional,  PavedDrive,  SaleType,  SaleCondition


**3. Train/Validation/Test split**
* Split the data into train and test set. (75-25 split ratio)
* Split the training set into training and validation sets. For the entire dataset we did a 60-15-25 split.

In [15]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size = 0.25, random_state = 18)

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.2, random_state = 18)

**4. Feature separation train/validation set (numerical,categorical)**

In [17]:
X_train_numerical = X_train[numerical_features]

In [18]:
print("The X_train_numerical dataset contains {} rows and {} columns".format(X_train_numerical.shape[0],X_train_numerical.shape[1]))

The X_train_numerical dataset contains 870 rows and 34 columns


In [19]:
X_train_categorical = X_train[categorical_features]

In [20]:
print("The X_train_categorical dataset contains {} rows and {} columns".format(X_train_categorical.shape[0],X_train_categorical.shape[1]))

The X_train_categorical dataset contains 870 rows and 29 columns


In [21]:
X_valid_numerical = X_valid[numerical_features]

In [22]:
print("The X_valid_numerical dataset contains {} rows and {} columns".format(X_valid_numerical.shape[0],X_valid_numerical.shape[1]))

The X_valid_numerical dataset contains 218 rows and 34 columns


In [23]:
X_valid_categorical = X_valid[categorical_features]

In [24]:
print("The X_valid_categorical dataset contains {} rows and {} columns".format(X_valid_categorical.shape[0],X_valid_categorical.shape[1]))

The X_valid_categorical dataset contains 218 rows and 29 columns


**5. Maintain the same set of levels across the training and validation set for categorical features.**
* The number of categorical features at the same level get reduced from 29 features to only 12 features.

In [25]:
keep = X_train_categorical.nunique() == X_valid_categorical.nunique()
X_train_categorical = X_train_categorical[X_train_categorical.columns[keep]]
X_valid_categorical = X_valid_categorical[X_valid_categorical.columns[keep]]

In [26]:
keep = list()
for i in range(X_train_categorical.shape[1]):
    keep.append(all(np.sort(X_train_categorical.iloc[:,i].unique()) == np.sort(X_valid_categorical.iloc[:,i].unique())))
X_train_categorical = X_train_categorical[X_train_categorical.columns[keep]]
X_valid_categorical = X_valid_categorical[X_valid_categorical.columns[keep]]

In [27]:
print("The X_train_categorical dataset after running the above code now contains {} rows and {} columns".format(X_train_categorical.shape[0],X_train_categorical.shape[1]))

The X_train_categorical dataset after running the above code now contains 870 rows and 12 columns


In [28]:
print("The X_valid_categorical dataset after running the above code contains {} rows and {} columns".format(X_valid_categorical.shape[0],X_valid_categorical.shape[1]))

The X_valid_categorical dataset after running the above code contains 218 rows and 12 columns


**6. Hyperparameter tuning**  
To perform PCA and MCA, we are using the methods provided by **[prince](https://github.com/MaxHalford/prince)** library and will tune the same set of parameters for both PCA and MCA. Our dataset has around 1500 observations and we chose only a few parameters as the number of resulting combinations to check were way more than the number of observations in the dataset. Still we ended up with 4000+ combinations. Below mentioned are the parameters that will be tuned:

1. Number of components (n_components): It denotes the number of components used to perform PCA or MCA.
2. Number of iterations (n_iter): The number of iterations used for computing the SVD.

To standardize the numerical data we will be setting the **rescale_with_mean** and **rescale_with_std parameters** to True.  

As we are tuning parameters for PCA and MCA, we are also going to tune the alpha value for the ridge regression to determine the best alpha value for the specific parameter pair.  
If we get the lowest validation MSE for the pair, then we can use the best value for alpha that we determined while hyperparameter tuning.

In [29]:
n_components_pca = range(1,34) 
n_components_mca = range(1,12) # there are only 29 categorical features.
n_iter = range(1,6)
# hyperparameter pairs
hyperparameter_pairs = list(itertools.product(n_components_pca, n_components_mca, n_iter))
print("The total number of hyperparameter pairs for PCA and MCA are: {}".format(len(hyperparameter_pairs)))

The total number of hyperparameter pairs for PCA and MCA are: 1815


In [30]:
start = time.time() # Capture the execution start time.
Validation_Scores = list()
Best_Alphas = list()
for pair in hyperparameter_pairs:
    # Perform PCA
    PCA = prince.PCA(n_components = pair[0],
                     n_iter = pair[2],
                     rescale_with_mean=True, # Standardization
                     rescale_with_std=True, # Standardization
                     copy=True, # Prevent inplace substitution
                     check_input=True,
                     engine = 'sklearn', # sklearn engine
                     random_state = 18)
    # Fit only the training set.
    PCA.fit(X_train_numerical)
    # Transform both the training and validation set.
    X_train_numerical_transformed = PCA.transform(X_train_numerical)
    X_valid_numerical_transformed = PCA.transform(X_valid_numerical)
    
    # Perform MCA
    MCA = prince.MCA(n_components = pair[1],
                     n_iter = pair[2],
                     copy=True,
                     check_input=True,
                     engine='sklearn',
                     random_state=18)
    # Fit only the training set
    MCA.fit(X_train_categorical)
    # Transform both the training and validation set
    X_train_categorical_transformed = MCA.transform(X_train_categorical)
    X_valid_categorical_transformed = MCA.transform(X_valid_categorical)
    
    # Create a combined training and validation set containing reduced features.
    X_train_combined = pd.concat([X_train_numerical_transformed, X_train_categorical_transformed],axis = 1)
    X_valid_combined = pd.concat([X_valid_numerical_transformed, X_valid_categorical_transformed],axis = 1)
    
    # Tuning the alphas for the ridge
    alphas = np.logspace(-10, 10, 21) # We will use lambda on powers of 10 scale
    alphas_index = np.linspace(-10, 10, 21) # The lambda values on log scale
    # validation scores for alphas
    alpha_validation_score = []
    for a in alphas:
        ridge = Ridge(alpha=a,random_state=18)
        ridge.fit(X_train_combined, y_train)
        alpha_validation_score.append(metrics.mean_squared_error(ridge.predict(X_valid_combined), y_valid))
        
    best_alpha = alphas[np.argmin(alpha_validation_score)]
    Best_Alphas.append(best_alpha)
    
    # Fitting the model using the best alpha value to calculate the validation MSE.
    RIDGE = Ridge(alpha=best_alpha,random_state=18)
    RIDGE.fit(X_train_combined, y_train)
    Validation_Scores.append(metrics.mean_squared_error(RIDGE.predict(X_valid_combined), y_valid))
    
end = time.time() # Capture the execution end time.

In [31]:
print("Total execution time for hyperparameter tuning is: {:.4} mins".format(float(end-start)/60))

Total execution time for hyperparameter tuning is: 14.96 mins


In [32]:
print('The lowest MSE on the validation set is: {}'.format(min(Validation_Scores)))

The lowest MSE on the validation set is: 773388047.0924511


**7. Best parameters**

In [33]:
print('Best hyperparameter pair for PCA and MCA are {} and the best alpha value for Ridge Regression is: {}'.\
      format(hyperparameter_pairs[np.argmin(Validation_Scores)],Best_Alphas[np.argmin(Validation_Scores)]))

Best hyperparameter pair for PCA and MCA are (8, 11, 1) and the best alpha value for Ridge Regression is: 1e-10


**8. Reapplying the code on the (train+validation) and test set.**  
* Performing the same steps to separate the numerical and categorical features from the training+validation and test set.
* Maintaining the same levels for categorical features in the training+validation and test set.
* Using the best parameters to perform PCA on numerical features and MCA on categorical features.
* Combining the reduced numerical and categorical features to run Ridge regression using the best alpha value.
* Calculating the validation MSE on the test set.

In [31]:
X_train_valid_numerical = X_train_valid[numerical_features]

In [32]:
print("The X_train_valid_numerical dataset contains {} rows and {} columns".format(X_train_valid_numerical.shape[0],X_train_valid_numerical.shape[1]))

The X_train_valid_numerical dataset contains 1088 rows and 34 columns


In [33]:
X_train_valid_categorical = X_train_valid[categorical_features]

In [34]:
print("The X_train_valid_categorical dataset contains {} rows and {} columns".format(X_train_valid_categorical.shape[0],X_train_valid_categorical.shape[1]))

The X_train_valid_categorical dataset contains 1088 rows and 29 columns


In [35]:
X_test_numerical = X_test[numerical_features]

In [36]:
print("The X_test_numerical dataset contains {} rows and {} columns".format(X_test_numerical.shape[0],X_test_numerical.shape[1]))

The X_test_numerical dataset contains 363 rows and 34 columns


In [37]:
X_test_categorical = X_test[categorical_features]

In [38]:
print("The X_test_categorical dataset contains {} rows and {} columns".format(X_test_categorical.shape[0],X_test_categorical.shape[1]))

The X_test_categorical dataset contains 363 rows and 29 columns


In [39]:
# Maintaing the same levels across training+validation and test categorical features.
keep2 = X_train_valid_categorical.nunique() == X_test_categorical.nunique()
X_train_valid_categorical = X_train_valid_categorical[X_train_valid_categorical.columns[keep2]]
X_test_categorical = X_test_categorical[X_test_categorical.columns[keep2]]

In [40]:
keep2 = list()
for i in range(X_train_valid_categorical.shape[1]):
    keep2.append(all(np.sort(X_train_valid_categorical.iloc[:,i].unique()) == np.sort(X_test_categorical.iloc[:,i].unique())))
X_train_valid_categorical = X_train_valid_categorical[X_train_valid_categorical.columns[keep2]]
X_test_categorical = X_test_categorical[X_test_categorical.columns[keep2]]

In [41]:
print("The X_train_valid_categorical dataset after running the above code now contains {} rows and {} columns".format(X_train_valid_categorical.shape[0],X_train_valid_categorical.shape[1]))

The X_train_valid_categorical dataset after running the above code now contains 1088 rows and 16 columns


In [42]:
print("The X_test_categorical dataset after running the above code now contains {} rows and {} columns".format(X_test_categorical.shape[0],X_test_categorical.shape[1]))

The X_test_categorical dataset after running the above code now contains 363 rows and 16 columns


In [43]:
# Perform PCA
PCA = prince.PCA(n_components = hyperparameter_pairs[np.argmin(Validation_Scores)][0],
                     n_iter = hyperparameter_pairs[np.argmin(Validation_Scores)][2],
                     rescale_with_mean=True, # Standardization
                     rescale_with_std=True, # Standardization
                     copy=True, # Prevent inplace substitution
                     check_input=True,
                     engine = 'sklearn', # sklearn engine
                     random_state = 18)
    
# Fit only the training set.
PCA.fit(X_train_valid_numerical)
# Transform both the training and validation set.
X_train_valid_numerical_transformed = PCA.transform(X_train_valid_numerical)
X_test_numerical_transformed = PCA.transform(X_test_numerical)

In [44]:
# Perform MCA
MCA = prince.MCA(n_components = hyperparameter_pairs[np.argmin(Validation_Scores)][1],
                     n_iter = hyperparameter_pairs[np.argmin(Validation_Scores)][2],
                     copy=True,
                     check_input=True,
                     engine='sklearn',
                     random_state=18)    
# Fit only the training set
MCA.fit(X_train_valid_categorical)
# Transform both the training and validation set
X_train_valid_categorical_transformed = MCA.transform(X_train_valid_categorical)
X_test_categorical_transformed = MCA.transform(X_test_categorical)

In [45]:
# Create a combined training and validation set containing reduced features.
X_train_valid_combined = pd.concat([X_train_valid_numerical_transformed, X_train_valid_categorical_transformed],axis = 1)
X_test_combined = pd.concat([X_test_numerical_transformed, X_test_categorical_transformed],axis = 1)

In [46]:
# Run RIDGE regression
RIDGE = Ridge(alpha=Best_Alphas[np.argmin(Validation_Scores)],random_state=18)

In [47]:
RIDGE.fit(X_train_valid_combined, y_train_valid)
print("Validation MSE on the test set is: {}".format(metrics.mean_squared_error(RIDGE.predict(X_test_combined), y_test)))

Validation MSE on the test set is: 738752163.8196424


**9. Baseline Ridge Regression**
* Comparing the above results (regression with reduced features) with the regression using the original dataset.

In [48]:
# Training set
X_train_dummy = pd.get_dummies(X_train_categorical,drop_first = True)

In [49]:
X_train_baseline_combined = pd.concat([X_train_numerical,X_train_dummy], axis=1)

In [50]:
print("The X_train_baseline_combined dataset contains {} rows and {} columns".format(X_train_baseline_combined.shape[0],X_train_baseline_combined.shape[1]))

The X_train_baseline_combined dataset contains 870 rows and 66 columns


In [51]:
# Validation set
X_valid_dummy = pd.get_dummies(X_valid_categorical,drop_first = True)

In [52]:
X_valid_baseline_combined = pd.concat([X_valid_numerical,X_valid_dummy], axis=1)

In [53]:
print("The X_valid_baseline_combined dataset contains {} rows and {} columns".format(X_valid_baseline_combined.shape[0],X_valid_baseline_combined.shape[1]))

The X_valid_baseline_combined dataset contains 218 rows and 66 columns


In [54]:
# Tuning the alphas for Ridge Regression
alphas = np.logspace(-10, 10, 21) # We will use lambda on powers of 10 scale
baseline_validation_scores = []
for a in alphas:
    lm = Ridge(alpha=a)
    scaler = StandardScaler() # standardize the data.
    scaler.fit(X_train_baseline_combined)
    X_train_baseline_combined_scaled = scaler.transform(X_train_baseline_combined)
    X_valid_baseline_combined_scaled = scaler.transform(X_valid_baseline_combined)
    lm.fit(X_train_baseline_combined_scaled, y_train) # Fit model on training set
    baseline_validation_scores.append(metrics.mean_squared_error(lm.predict(X_valid_baseline_combined_scaled), y_valid)) # Evaluate model on validation set

In [55]:
print('The lowest MSE on the validation set is: {}'.format(min(baseline_validation_scores)))

The lowest MSE on the validation set is: 896261228.2120639


In [56]:
print("The best value of alpha for the Ridge regression is: {}".format(alphas[np.argmin(baseline_validation_scores)]))

The best value of alpha for the Ridge regression is: 100.0


**10. Evaluation on the test set**

In [57]:
X_train_valid_dummy = pd.get_dummies(X_train_valid_categorical,drop_first = True)

In [58]:
X_train_valid_baseline_combined = pd.concat([X_train_valid_numerical,X_train_valid_dummy], axis=1)

In [59]:
print("The X_train_valid_baseline_combined dataset contains {} rows and {} columns".format(X_train_valid_baseline_combined.shape[0],X_train_valid_baseline_combined.shape[1]))

The X_train_valid_baseline_combined dataset contains 1088 rows and 108 columns


In [60]:
X_test_dummy = pd.get_dummies(X_test_categorical,drop_first = True)

In [61]:
X_test_baseline_combined = pd.concat([X_test_numerical,X_test_dummy], axis=1)

In [62]:
print("The X_test_baseline_combined dataset contains {} rows and {} columns".format(X_test_baseline_combined.shape[0],X_test_baseline_combined.shape[1]))

The X_test_baseline_combined dataset contains 363 rows and 108 columns


In [63]:
# Scale the data
scaler = StandardScaler()
scaler.fit(X_train_valid_baseline_combined)
X_train_valid_baseline_combined_scaled = scaler.transform(X_train_valid_baseline_combined)
X_test_baseline_combined_scaled = scaler.transform(X_test_baseline_combined)

# Fit the data
lm2 = Ridge(alpha=alphas[np.argmin(baseline_validation_scores)],random_state=18)
lm2.fit(X_train_valid_baseline_combined_scaled,y_train_valid)

# Validation MSE
print("Validation MSE on the test set is: {}".format(metrics.mean_squared_error(lm2.predict(X_test_baseline_combined_scaled), y_test)))

Validation MSE on the test set is: 716644289.4678497


**Dimensionality Reduction Summary**:  
We performed dimensionality reduction on the dataset containing both numerical as well as categorical features and tuned the parameters to perform regression. The below mentioned table shows the validation and test MSE for the Ridge regression using reduced features and original baseline dataset.

|MSE |Ridge Baseline |Ridge Reduced Features|
|:-:|:-:|:-:|
|Validation Set |896261228.21 |765992669.03 |
|Test Set | 716644289.46 | 738752163.81|


From the above table we can see that the MSE on the validation set was much lower for Ridge regression model with reduced features as compared to baseline Ridge regression model. However the performance of both the models on the test set is comparable.  
We got lower MSE on the test set for baseline Ridge regression model as compared to the Ridge regression model having reduced features.

### Part II: Clustering Analysis
* Performing clustering analysis with numerical and categorical data combined.

**1. Initial data analysis**

In [64]:
# Making a copy of the dataset
clustering_df = data.copy()

In [65]:
# Defining the predictors
X2 = data.drop(columns='SalePrice',axis=1)

In [66]:
X2.head(2)

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,706,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,2,548,Y,0,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,978,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,2,460,Y,298,0,0,0,0,0,0,5,2007,WD,Normal


**2. Assign labels**
* Dividing the dataframe into **five** clusters by assigning each row a label between 1 and 5.
* Using pd.qcut to divide (bin the response variable) the df into clusters.

In [67]:
clustering_df['Clusters'] = pd.qcut(clustering_df.SalePrice, q = 5, labels = [1, 2, 3, 4,5])

In [68]:
clustering_df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Clusters
0,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,706,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,2,548,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500,4
1,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,978,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,2,460,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500,4
2,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,486,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,2,608,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500,4
3,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,216,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,3,642,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000,2
4,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,655,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,3,836,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000,5


In [69]:
# Checking the counts
clustering_df.groupby(clustering_df.Clusters).count()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Clusters,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295,295
2,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294,294
3,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284,284
4,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291,291
5,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287,287


**3. Compute Gower distance**
* Create an input matrix for the K-medoids clustering algorithm.
* Need to install [gower](https://pypi.org/project/gower/) package before running the next cell.

In [70]:
# Gower distance
matrix = gower.gower_matrix(X2)

**4. K-medoids clustering**
* Using the methods provided by pyclustering package. Need to install it before running the next cell.
* Set random initial medoids.
* Run k-medoids for processing distance matrix instead of points.
* Perform cluster analysis to get results.
* Create clusters and medoids.

In [71]:
# random medoids
initial_medoids = np.random.randint(1,1451, 5)

In [72]:
# kmedoids
kmedoids_instance = kmedoids(matrix, initial_medoids, data_type='distance_matrix')

In [73]:
# Cluster analysis
kmedoids_instance.process()

In [74]:
# Get clusters
clusters = kmedoids_instance.get_clusters()

In [75]:
# Count the number of observations in the new clusters.
for i in range(len(clusters)):
    print('Number of observations in cluster {} is {}'.format(i+1,len(clusters[i])))

Number of observations in cluster 1 is 61
Number of observations in cluster 2 is 82
Number of observations in cluster 3 is 620
Number of observations in cluster 4 is 249
Number of observations in cluster 5 is 439


In [76]:
# Get updated medoids
medoids = kmedoids_instance.get_medoids()

**5. Map the newly created labels to the existing data**
* Get the corresponding labels from the clusters and store it in the list.
* Add the list to the clustering_df to perform comparision.

In [77]:
# Get the corresponding labels and create a list of labels.
labels = list()
for i in range(len(clustering_df.index)):
    if i in clusters[0]:
        labels.append(1) # cluster 1
    elif i in clusters[1]:
        labels.append(2)
    elif i in clusters[2]:
        labels.append(3)
    elif i in clusters[3]:
        labels.append(4)
    elif i in clusters[4]:
        labels.append(5)


In [78]:
# Adding the list as new column to the dataframe.
clustering_df['NewClusters'] = labels

In [79]:
clustering_df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Clusters,NewClusters
0,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,706,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,2,548,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500,4,5
1,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,978,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,2,460,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500,4,3
2,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,486,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,2,608,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500,4,5
3,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,216,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,3,642,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000,2,2
4,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,655,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,3,836,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000,5,5


**6. Calculating the normalized mutual information (NMI)**
* Compute the normalized mutual information (NMI) between your clustering results and the binned categories.

In [80]:
print("The NMI score between the clustering results and the binned categories is: {:.4}".format(normalized_mutual_info_score(clustering_df['Clusters'],clustering_df['NewClusters'])))

The NMI score between the clustering results and the binned categories is: 0.2458


**Clustering Analysis Summary**:  
We binned the response variable 'SalePrice' into 5 bins which contain almost same number of observations.  
Our NMI score is very low i.e. 0.24 which clearly indicates that the cluster assignment was only partially correct.  
Since, the observations got assigned to clusters disproportionately and cluster 3 (620) and cluster 5 (439) got the major chunk of the observations the low NMI score is understandable.